## Benchmark for Reading and Datamining PDB Structures with mmtf-pyspark

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.filters import ContainsGroup
from mmtfPyspark.utils import ColumnarStructure
from mmtfPyspark.interactions import InteractionExtractorPd

import gzip
import pandas as pd
import numpy as np
import os
import time

## Setup the benchmark
Set the path to the MMTF Hadoop Sequence file. Here we retrieve the value of the environment variable MMTF_FULL

In [2]:
path = mmtfReader.get_mmtf_full_path()

Hadoop Sequence file path: MMTF_FULL=/Users/peter/MMTF_Files/full


Specify a list with the number of cores

In [3]:
cores = [4]

In [4]:
# create results directory
results_dir = '../results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

## Structure Benchmark
This benchmark read structures

In [13]:
def structure(path, num_cores):
    spark = SparkSession.builder.master("local[" + str(num_cores) + "]").appName("Benchmark3").getOrCreate()
    structures = mmtfReader.read_sequence_file(path, first_model=True)                
    count = structures.count()

    spark.stop()
    return count

In [14]:
df_s = pd.DataFrame(columns=('cores', 'structures'))

for num_cores in cores:
    start = time.time()
    count = structure(path, num_cores)
    end = time.time()
    print('structures, cores:', num_cores, 'time:', end-start, 'seconds')
    df_s = df_s.append([{'cores':num_cores, 'structures': end-start, 'count': count}], ignore_index=True, sort=True)

structures, cores: 4 time: 204.89131379127502 seconds


In [15]:
df_s.to_csv(os.path.join(results_dir, 'structures.csv'), index=False)

In [16]:
df_s

,cores,count,structures
0,4,140825.0,204.891314


## Structure calc_core_group_data Benchmark


In [5]:
def structure(path, num_cores):
    spark = SparkSession.builder.master("local[" + str(num_cores) + "]").appName("Benchmark3").getOrCreate()
    structures = mmtfReader.read_sequence_file(path, first_model=True)
    structures.foreach(lambda s: s[1].calc_core_group_data())
    count = 0

    spark.stop()
    return count

In [6]:
df_s = pd.DataFrame(columns=('cores', 'structures_group_data'))

for num_cores in cores:
    start = time.time()
    count = structure(path, num_cores)
    end = time.time()
    print('structures, cores:', num_cores, 'time:', end-start, 'seconds')
    df_s = df_s.append([{'cores':num_cores, 'structures_group_data': end-start, 'count': count}], ignore_index=True, sort=True)

structures, cores: 4 time: 625.5085597038269 seconds


In [7]:
df_s.to_csv(os.path.join(results_dir, 'structures_group_data.csv'), index=False)

In [8]:
df_s

,cores,count,structures_group_data
0,4,0.0,625.50856


## Structure To Pandas Benchmark
This benchmark read structures and converts them to pandas dataframes
default: 891, 819, 855 (no JIT), 1039 (cache=True)
use_categories: 1079
multi_index: 

In [17]:
def structure(path, num_cores):
    spark = SparkSession.builder.master("local[" + str(num_cores) + "]").appName("Benchmark3").getOrCreate()
    structures = mmtfReader.read_sequence_file(path, first_model=True)
    dfs = structures.map(lambda s: s[1].to_pandas())
    count = dfs.count()

    spark.stop()
    return count

In [18]:
df_s = pd.DataFrame(columns=('cores', 'structures_to_pandas'))

for num_cores in cores:
    start = time.time()
    count = structure(path, num_cores)
    end = time.time()
    print('structures, cores:', num_cores, 'time:', end-start, 'seconds')
    df_s = df_s.append([{'cores':num_cores, 'structures_to_pandas': end-start, 'count': count}], ignore_index=True, sort=True)

structures, cores: 4 time: 823.9224970340729 seconds


In [19]:
df_s.to_csv(os.path.join(results_dir, 'structures_to_pandas.csv'), index=False)

In [20]:
df_s

,cores,count,structures_to_pandas
0,4,140825.0,823.922497


## Structure to Chain Benchmark
This benchmark read structures and flatmaps to polymer chains

In [13]:
def structure_to_chains(path, num_cores):
    spark = SparkSession.builder.master("local[" + str(num_cores) + "]").appName("Interactions").getOrCreate()
    structures = mmtfReader.read_sequence_file(path)
    chains = structures.flatMap(lambda s: s[1].get_chains())                 
    count = chains.count()

    spark.stop()
    return count

In [14]:
df_s2c = pd.DataFrame(columns=('cores', 'structure_to_chains'))

for num_cores in cores:
    start = time.time()
    count = structure_to_chains(path, num_cores)
    end = time.time()
    print('structure_to_chains, cores:', num_cores, 'time:', end-start, 'seconds')
    df_s2c = df_s2c.append([{'cores':num_cores, 'structure_to_chains': end-start, 'count': count}], ignore_index=True, sort=True)

structure_to_chains, cores: 4 time: 243.24330711364746 seconds


In [15]:
df_s2c.to_csv(os.path.join(results_dir, 'structure_to_chains.csv'), index=False)

In [16]:
df_s2c

,cores,count,structure_to_chains
0,4,447250.0,243.243307


## Structure to Chain to pandas Benchmark
This benchmark read structures and flatmaps to polymer chains and convert to pandas dataframes

In [17]:
def structure_to_chains(path, num_cores):
    spark = SparkSession.builder.master("local[" + str(num_cores) + "]").appName("Interactions").getOrCreate()
    structures = mmtfReader.read_sequence_file(path)
    chains = structures.flatMap(lambda s: s[1].get_chains())
    dfc = chains.map(lambda c: c.to_pandas())
    count = dfc.count()

    spark.stop()
    return count

In [18]:
df_s2c = pd.DataFrame(columns=('cores', 'structure_to_chains'))

for num_cores in cores:
    start = time.time()
    count = structure_to_chains(path, num_cores)
    end = time.time()
    print('structure_to_chains, cores:', num_cores, 'time:', end-start, 'seconds')
    df_s2c = df_s2c.append([{'cores':num_cores, 'structure_to_chains': end-start, 'count': count}], ignore_index=True, sort=True)

structure_to_chains, cores: 4 time: 1272.5538392066956 seconds


In [19]:
df_s2c.to_csv(os.path.join(results_dir, 'structure_to_chains_to_pandas.csv'), index=False)

In [20]:
df_s2c

,cores,count,structure_to_chains
0,4,447250.0,1272.553839


## Saltbridges Benchmark
This benchmark finds salt bridges in protein structures. Structures with multiple models, e.g., NMR structures are excluded.

In [5]:
def saltbridges(path, num_cores):
    spark = SparkSession.builder.master("local[" + str(num_cores) + "]").appName("Saltbridges").getOrCreate()
    structures = mmtfReader.read_sequence_file(path, fraction=0.05)
    structures = structures.filter(lambda s: s[1].num_models == 1)
                               
    distance_cutoff = 3.5
    query = "polymer and (group_name in ['ASP', 'GLU']) and (atom_name in ['OD1', 'OD2', 'OE1', 'OE2'])"
    target = "polymer and (group_name in ['ARG', 'LYS', 'HIS']) and (atom_name in ['NH1', 'NH2', 'NZ', 'ND1', 'NE2'])"

    interactions = InteractionExtractorPd.get_interactions(structures, distance_cutoff, query, target, bio=None)
    count = interactions.count()

    spark.stop()
    return count

In [6]:
df_saltbridges = pd.DataFrame(columns=('cores', 'saltbridges_pd'))

for num_cores in cores:
    start = time.time()
    count = saltbridges(path, num_cores)
    end = time.time()
    print('saltbridges_pd, cores:', num_cores, 'time:', end-start, 'seconds')
    df_saltbridges = df_saltbridges.append([{'cores':num_cores, 'saltbridges_pd': end-start, 'count': count}], ignore_index=True, sort=True)

Py4JJavaError: An error occurred while calling o60.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 1.0 failed 1 times, most recent failure: Lost task 1.0 in stage 1.0 (TID 2, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/anaconda3/envs/mmtf-pyspark-benchmarks-dev/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/anaconda3/envs/mmtf-pyspark-benchmarks-dev/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/anaconda3/envs/mmtf-pyspark-benchmarks-dev/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/anaconda3/envs/mmtf-pyspark-benchmarks-dev/lib/python3.7/site-packages/py

In [ ]:
df_saltbridges.to_csv(os.path.join(results_dir, 'saltbridges_pd.csv'), index=False)
df_saltbridges
